In [1]:
from plot
import pandas as pd
import warnings
import os
import gc
pd.set_option('display.max_columns', 1_00)
warnings.filterwarnings('ignore')

In [2]:
RAW_DATA_PATH_ADMISSION = '../raw_data/IDs_mapping.csv'
RAW_DATA_PATH_DIABETES  = '../raw_data/diabetic_data.csv'
DATA_PATH = '../data'
DATA_PATH_MAP = DATA_PATH + '/map_id'

# 1. IMPORT

In [3]:
df_map = pd.read_csv(RAW_DATA_PATH_ADMISSION)
df = pd.read_csv(RAW_DATA_PATH_DIABETES)

# 2. TIDY

### 2.1. Add descriptions to main dataframe `df` and remove ids

In [4]:
# admission types
df_admission = df_map[:8]
df_discharge = df_map[10:40].rename(columns = {'admission_type_id':'discharge_disposition_id'})
df_admission_source = df_map[42:66].rename(columns = {'admission_type_id': 'admission_source_id'})
df_admission.iloc[:,0] = df_admission.iloc[:,0].astype(int)
df_discharge.iloc[:,0] = df_discharge.iloc[:,0].astype(int)
df_admission_source.iloc[:,0] = df_admission_source.iloc[:,0].astype(int)
df_admission.rename(columns = {'description': 'admission_type'},inplace=True)
df_discharge.rename(columns = {'description': 'discharge_disposition'},inplace=True)
df_admission_source.rename(columns = {'description': 'admission_source'},inplace=True)

# clean columns in df
df.columns = list(map(lambda i: i.lower().replace('-','_'), df.columns))

# merge with maps ids
df = df.merge(df_admission, on = 'admission_type_id', how='left')
df = df.merge(df_discharge, on = 'discharge_disposition_id', how='left')
df = df.merge(df_admission_source, on = 'admission_source_id', how='left')

# remove description columns
df.drop(['admission_type_id', 'discharge_disposition_id', 'admission_source_id'], axis=1, inplace=True)

# drop duplicates
df = df.drop_duplicates().reset_index(drop=True)

# save
df.to_csv(os.path.join(DATA_PATH, 'diabetes_tidy.csv'), index=False)

# free memory
del df_discharge, df_admission, df_admission_source
gc.collect()

0

### 2.2 Create a dataset in DATA_PATH_MAP with all map dataframe to map description and ids

In [5]:
def df_id_map(df, column):
    aux = pd.DataFrame(df[column].value_counts().index, columns=[column])
    aux[f'{column}_id'] = list(range(1,len(aux)+1))
    return aux

def create_maps(df, columns, data_path):
    map_dict = {i:df_id_map(df,i) for i in columns}
    # save map dict
    for k, v in map_dict.items():
        v.to_csv(os.path.join(data_path, f'{k}.csv'), index=False)

In [6]:
create_maps(df, list(df.select_dtypes('object').columns), DATA_PATH_MAP)

### 2.3. Apply the mapping system created to the main dataframe `df`

In [7]:
def apply_map_id(df, data_path):
    data_maps = list(map(lambda i: os.path.join(data_path,i),os.listdir(data_path)))
    for i in data_maps:
        aux = pd.read_csv(i)
        id_col = list(filter(lambda i:i.endswith('_id'), aux.columns))[0]
        on_col = list(set(aux.columns).difference(set([id_col])))[0]
        aux[id_col] = aux[id_col].astype(int)
        df = df.merge(aux,on=on_col, how='left')
        df.drop(on_col, axis=1, inplace = True)
        df[id_col] = df[id_col].fillna(0)
        df[id_col] = df[id_col].astype(int)
    return df

In [8]:
df_tidy = apply_map_id(df, DATA_PATH_MAP)

In [9]:
df_tidy.dtypes

encounter_id                   int64
patient_nbr                    int64
time_in_hospital               int64
num_lab_procedures             int64
num_procedures                 int64
num_medications                int64
number_outpatient              int64
number_emergency               int64
number_inpatient               int64
number_diagnoses               int64
rosiglitazone_id               int64
acetohexamide_id               int64
a1cresult_id                   int64
citoglipton_id                 int64
age_id                         int64
gender_id                      int64
glipizide_metformin_id         int64
miglitol_id                    int64
metformin_id                   int64
tolbutamide_id                 int64
nateglinide_id                 int64
acarbose_id                    int64
admission_type_id              int64
pioglitazone_id                int64
weight_id                      int64
readmitted_id                  int64
troglitazone_id                int64
g

In [10]:
# save it
df_tidy.to_csv(os.path.join(DATA_PATH, 'diabetes_integers.csv'), index=False)

# 3. Explore

### 3.1 Remove some columns with high missing values [`weight`, `payer_code`, `medical_specialty`]

In [24]:
missing_values = {i : (df[i] == '?').mean() for i  in df.columns}
missing_values = {k:v for k,v in missing_values.items() if v>0}
pd.DataFrame.from_dict([missing_values]).melt()

,variable,value
0,race,0.022336
1,weight,0.968585
2,payer_code,0.395574
3,medical_specialty,0.490822
4,diag_1,0.000206
5,diag_2,0.003518
6,diag_3,0.013983


In [26]:
df_tidy = df_tidy.drop(['weight_id', 'payer_code_id', 'medical_specialty_id'],axis=1,inplace=True)

### 3.2 Categorical values
As we can see in the next picture, the univariant effeect of categorical variables has no impact to clasiffy an observation as readmitted or not. The frequentist analysis of those categories shows that the percentage of each value of each category are quite close to be 50%

<img src="../img/categorical_univariant_effect.png" width=800 height=800 />


### 3.2 Numerical values
Those density plots are easy to explain. the variables have negative imapact, I mean, the more the worse, the more medication yo need, the more prob. to be readmitted, the more time in hostpital, the more prob. to be readmitted. 
So when the X increase the density graph green (be readmitted) is higher than de density of red (not). 
But as we can see those numerical variables has not much power to classify

<img src="../img/density_numerics.png" width=800 height=800 />


### 3.2 High cardinality categories
When categorical variables have many categories, we call them high cardinality categorical variables. It is usually difficult to work with them, a trick is to find that category that hides more information, that is, that has more predictive capacity. For this we can use the standard deviation in the frequency of appearance in readmitted (1) or not (0) and we see that there are certain categories with a lot of classification capacity. Good news. 


<img src="../img/high_categorical.png" width=800 height=800 />
